# Worst Fit

#### Set path

In [ ]:
# Define paths
REPO_PATH = r"C:\Users\Andre\projects\v1_v4_1024_elec" # set this
PROJECT_PATH = r'G:\Shared drives\v1_v4_1024'

#### Imports

In [ ]:
# general
import numpy as np
import matplotlib.pyplot as plt
import os
import odml

# custom
import sys
sys.path.append(REPO_PATH)

#### Settings

In [ ]:
# Session of interest (will ultimately apply to all sessions)
SESSIONS = ["A_SNR_041018", "A_SNR_140819", "L_SNR_250717"]
SESSION = "A_SNR_140819"

In [ ]:
# Dataset details
FS = 500 # sampling frequency

#### Functions

## Main

#### Create output directory

In [ ]:
# dir_out = f"{PROJECT_PATH}/figures/"
# if not os.path.exists(dir_out):
#     os.makedirs(dir_out)

#### Load data

In [ ]:
# # define odml file path (contains metadata including electrode positions)
# odml_path = fr"{PROJECT_PATH}\data\metadata\{SESSION}\metadata_{SESSION}.odml"
# metadata = odml.load(odml_path)